# Model training notebook


In [ ]:
from LangDataloader import LangDataloader
from LangDataloader import addData
import numpy as np
import tensorflow as tf
import keras_lmu 
import copy
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
defultePlotSize = plt.rcParams['figure.figsize']
oringalFontSize = matplotlib.rcParams['font.size']

In [ ]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

In [ ]:
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.compat.v1.Session(config=config)
# sess.as_default()

We will only load a subset of the total dataset to save time. 

In [ ]:
data = LangDataloader("/home/khood/GitHub/STREAM/Prepocessing/processedData/index.csv", "/home/khood/GitHub/STREAM/Prepocessing/processedData")
rawData = []
lastStep = 0
labelToNum = {}
num = 0
for step in list(range(0,len(data),100)):
    for file in range(len(data))[lastStep:step]:
        print(f"adding {lastStep} to {step} of {len(data)}: {'{0:.0f}%'.format(step/len(data) * 100)}... ",end="\x1b\r")
        label = None
        if data[file][1][0] in labelToNum.keys():
            label = np.array([labelToNum[data[file][1][0]]])
        else:
            labelToNum[data[file][1][0]] = num
            num = num + 1
            label = np.array([labelToNum[data[file][1][0]]])
            
        melspectrogram = np.array([data[file][0]]).flatten()
        melspectrogram = [[i] for i in melspectrogram]
        rawData.append([melspectrogram, label])
        
    lastStep = step
        
# for file in range(len(data))[lastStep:]: # grab the remaning (if the size%stepSize != 0 then there will be some left over)
#     print(f"adding {lastStep} to {len(data)} of {len(data)}: {'{0:.0f}%'.format(step/len(data) * 100)}... ",end="\x1b\r")
#     rawData.append(data[file])
    
rawData[0][0]

convert targets to numbers classes

In [ ]:
len(labelToNum.keys())

The last dimension should be time, the second to last should be the n_mels

<p>not flattened shape: (1, 128, 430)
<p>flattened shape: (55040,)

In [ ]:
n_pixels = 55040

lmu_layer = keras_lmu.LMU(
    memory_d=1,
    order=256,
    theta=n_pixels,
    hidden_cell=tf.keras.layers.SimpleRNNCell(2750),
    hidden_to_memory=False,
    memory_to_memory=False,
    input_to_hidden=True,
    kernel_initializer="ones",
)

# TensorFlow layer definition
inputs = tf.keras.Input((n_pixels, 1))
lmus = lmu_layer(inputs)
outputs = tf.keras.layers.Dense(len(labelToNum.keys()))(lmus)

# TensorFlow model definition
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)
model.summary()

In [ ]:
valid = []
train = []
addData(valid, train, rawData, int(len(rawData)*0.9))
print(f"Train: {len(train)}")
print(f"Valid: {len(valid)}")

In [ ]:
X_train = []
Y_train = []
for i in train:
    X_train.append(np.array(i[0]))
    Y_train.append(i[0][0][0])
X_train=np.array(X_train)
Y_train=np.array(Y_train)

In [ ]:
print(f"Y_train size: {len(Y_train)}, X_train size: {len(X_train)}")
Y_train.shape

In [ ]:
Y_train[:10]

In [ ]:
print(X_train[0])
print(len(X_train[0]))
len(X_train)

In [ ]:
print(Y_train)
len(Y_train)

In [ ]:
X_valid = []
Y_valid = []
for i in valid:
    X_valid.append(i[0])
    Y_valid.append(i[0][0])
X_valid=np.array(X_valid)
Y_valid=np.array(Y_valid)

In [ ]:
do_training = True
batch_size = 10
epochs = 10

saved_weights_fname = "./psMNIST-weights.hdf5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=saved_weights_fname, monitor="val_loss", verbose=1, save_best_only=True
    ),
]

result = None
if do_training:
    result = model.fit(
        X_train,
        Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_valid, Y_valid),
        callbacks=callbacks,
    )